# Basics and Cartesian Reconstructions
Here we are going to have a look at a few basics of MRpro and reconstruct data acquired with a Cartesian sampling pattern.

## Overview

In this notebook we are going to explore the MRpro KData object and the included header parameters. We will then use a FFT-operator in order to reconstruct data acquired with a Cartesian sampling scheme. Finally we will try to reconstruct data acquired on a Cartesian grid but with partial echo and partial Fourier acceleration.

Run this notebook in Google colab: 

<a target="_blank" href="https://colab.research.google.com/github/PTB-MR/mrpro_utrecht_workshop_2024/blob/main/solution_basics_and_cartesian_reconstruction.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Install MRpro and download data

In [ ]:
# Install MRpro
!pip install git+https://github.com/PTB-MR/mrpro.git#egg=mrpro[notebook]
import mrpro

In [ ]:
# Get the raw data from zenodo
import tempfile
from pathlib import Path
import zenodo_get

data_folder = Path(tempfile.mkdtemp())
dataset = '14173489'
zenodo_get.zenodo_get([dataset, '-r', 5, '-o', data_folder])  # r: retries

In [ ]:
# List the downloaded files
from os import listdir
for f in listdir(data_folder):
    print(f)

So we have three different scans obtained from the same object with the same FOV and resolution:

- cart_t1.mrd is a fully sampled Cartesian acquisition

- cart_t1_partial_echo_partial_fourier.mrd is accelerated using partial echo and partial Fourier

- cart_t1_msense_integraded.mrd is acclerated using regular undersampling and self-calibrated SENSE

## Read in raw data and explore header

To read in a ISMRMRD raw data file we can simply pass on the file name to a KData object. In addition we also have to provide information about the trajectory. In MRpro this is done using trajectory calculators, which are functions that calculate the trajectory based on the acquisition information and additional parameters provided to the calculators (e.g. the angular step for a radial acquisition). In this case we have a Cartesian acquisition so we only need to provide a Cartesian trajectory calculator without any further parameters.

In [ ]:
kdata = mrpro.data.KData.from_file(data_folder / Path('cart_t1.mrd'), mrpro.data.traj_calculators.KTrajectoryCartesian())

Now we can explore this data object.

In [ ]:
# Start with simply calling print(kdata)
print(kdata)

In [ ]:
# We can also have a look at the content of the header
print(kdata.header.acq_info.position[0,0,0,0])

## Reconstruction of fully sampled acquisition

We have got a fully sampled Cartesian acquisition so we know we can use a Fast Fourier Transform (FFT) to reconstruction the data. Let's create an FFT-operator and apply it to kdata. Here it is important to note that all MRpro operators work on PyTorch tensors and not on the MRpro objects directly. Therefore we have to call the operator on kdata.data. One other important feature of MRpro operators is that the always return a tuple of length 1 of PyTorch tensors.

In [ ]:
fft_op = mrpro.operators.FastFourierOp(dim=(-2,-1))
idat = fft_op.adjoint(kdata.data)[0]

Let's have a look at the shape of the obtained tensor.

In [ ]:
print(idat.shape)

We can see that the second dimension which is the coil dimension is 16, so we still have a coil resolved dataset. We can use a simply root-sum-of-squares approach to combine them into one. Later we will do something a bit more sophisticated. We can also see that the x-dimension is 512. This is because in MRI we commonly oversample the readout direction by a factor 2 leading to a FOV twice as large as we actually need. We can either remove this oversampling along the readout direction or we can simply tell the FFT-operator to remove it. 

In [ ]:
fft_op = mrpro.operators.FastFourierOp(dim=(-2,-1), recon_matrix=kdata.header.recon_matrix, encoding_matrix=kdata.header.encoding_matrix)
idat = fft_op.adjoint(kdata.data)[0]
print(idat.shape)

Now we have an image which is 256 x 256 voxel as we would expect. Let's combine the data from the different receiver coils using root-sum-of-squares and then display the image.

In [ ]:
import torch
idat = torch.sqrt(torch.sum(idat**2, dim=1)).abs()

import matplotlib.pyplot as plt
plt.figure()
plt.imshow(idat[0,0])


## Reconstruction of acquisition with partial echo and partial Fourier

Great! That was very easy so let's try to reconstruct the next dataset.

In [ ]:
kdata_pe_pf = mrpro.data.KData.from_file(data_folder / Path('cart_t1_partial_echo_partial_fourier.mrd'), mrpro.data.traj_calculators.KTrajectoryCartesian())
fft_op = mrpro.operators.FastFourierOp(dim=(-2,-1), recon_matrix=kdata.header.recon_matrix, encoding_matrix=kdata.header.encoding_matrix)
idat_pe_pf = fft_op.adjoint(kdata_pe_pf.data)[0]
idat_pe_pf = torch.sqrt(torch.sum(idat_pe_pf**2, dim=1)).abs()

fig, ax = plt.subplots(1,2, squeeze=False)
ax[0,0].imshow(idat[0,0])
ax[0,1].imshow(idat_pe_pf[0,0])


Well we get an image out but when we compare it to the previous result it seems the head as shrunk. As this is not very likely there is probably a mistake in our reconstruction. 

Alright let's take a step back and have a look at the trajectory of both scans.

In [ ]:
print(kdata.traj)

We can see that the trajectory consists of a kz, ky and kx part. Kx and ky only vary along a single dimension. The reason for this is that we try to save the trajectory in the most efficient way in MRpro. If we want to get the fully trajectory as a tensor we can call as_tensor().

In [ ]:
plt.figure()
plt.plot(kdata.traj.as_tensor()[2,0,0,:,:].flatten(), kdata.traj.as_tensor()[1,0,0,:,:].flatten(), 'ob')
plt.plot(kdata_pe_pf.traj.as_tensor()[2,0,0,:,:].flatten(), kdata_pe_pf.traj.as_tensor()[1,0,0,:,:].flatten(), '+r');

We can see that for the fully sampled acquisition the k-space is covered symmetrically from -256 to 255 along the readout direction and from -128 to 127 along the phase encoding direction. For the acquisition with partial Fourier and partial echo acceleration this is of course not the case and the k-space is asymetrical. 

Our FFT-operator does not know about this and simply assumes that the acquisition is symmetric and any difference between encoding and recon matrix need to be zero-padded symmetrically.

To take the asymetric acquisition into account and sort the data correctly into a matrix where we can apply the FFT-operator to, we have got the CartesianSamplingOp in MRpro. This operator calculates a sorting index based on the k-space trajectory and the dimensions of the encoding k-space.

In [ ]:
cart_sampling_op = mrpro.operators.CartesianSamplingOp(kdata_pe_pf.header.encoding_matrix, kdata_pe_pf.traj)

Now we can first apply the CartesianSamplingOp and then call the FFT-operator.

In [ ]:
idat_pe_pf = fft_op.adjoint(cart_sampling_op.adjoint(kdata_pe_pf.data)[0])[0]
idat_pe_pf = torch.sqrt(torch.sum(idat_pe_pf**2, dim=1)).abs()

fig, ax = plt.subplots(1,2, squeeze=False)
ax[0,0].imshow(idat[0,0])
ax[0,1].imshow(idat_pe_pf[0,0])

Voila! Now we get the same brains with the same size. But hang on a second - there is still something which looks a bit funny. In the bottom left hand corner it seems that there is a "hole" in the brain. This should probably not be there. 

The reason for this is, that we simply combined the data from the different coils using a root-sum-of-squares approach. This was easy but not what we should do. Commonly coil sensitivity maps are calculated and they are then used to combine the data from the different coils. In MRpro this is done by calculating coil sensitivity data and then creating a SensitivityOp to combine the data after image reconstruction.  

We have different option of how to calculate coil sensitivity maps from image data of the different coils. Here we are going to use the Walsh-method.

In [ ]:
# Calculate coil sensitivity maps
idat_pe_pf = fft_op.adjoint(cart_sampling_op.adjoint(kdata_pe_pf.data)[0])[0]
# This algorithms is designed to calculate coil sensitivity maps for each other dimension. 
csm_data = mrpro.algorithms.csm.walsh(idat_pe_pf[0,...], smoothing_width=5)[None,...]

# Create SensitivityOp
csm_op = mrpro.operators.SensitivityOp(csm_data)

# Reconstruct coil-combined image
idat_pe_pf = csm_op.adjoint(fft_op.adjoint(cart_sampling_op.adjoint(kdata_pe_pf.data)[0])[0])[0].abs()

fig, ax = plt.subplots(1,2, squeeze=False)
ax[0,0].imshow(idat[0,0])
ax[0,1].imshow(idat_pe_pf[0,0,0])

Now we got an image without any "holes"! 

When we reconstructed the image we called the adjoint method of several different operators one after the other. That was a bit cumbersome. To make our life easier we can combine the operators directly and then call the adjoint of the composite operator. We have to keep in mind that we have to put them in the order of the forward method of the operators. By calling the adjoint, the order will be automatically reversed.

In [ ]:
# Create composite operator
acq_op = cart_sampling_op @ fft_op @ csm_op 
idat_pe_pf = acq_op.adjoint(kdata_pe_pf.data)[0].abs()

fig, ax = plt.subplots(1,2, squeeze=False)
ax[0,0].imshow(idat[0,0])
ax[0,1].imshow(idat_pe_pf[0,0,0])

Although we have got now a nice looking image, it was a bit cumbersome to create it. We had to define several different operators and chain them together. Would we nice if this could be done automatically....

That is why we also included some top-level reconstruction algorithms. The reconstruction above we simply call DirectReconstruction. A DirectReconstruction object can be directly created from the information in the KData object.

Reconstruction algorithms operator on the data objects of MRpro, i.e. the input is a KData object and the output is a IData object. To visualise this we need the tensor content of the IData object which can be obtained by calling .rss().

In [ ]:
direct_recon_pe_pf = mrpro.algorithms.reconstruction.DirectReconstruction(kdata_pe_pf)
idat_pe_pf = direct_recon_pe_pf(kdata_pe_pf)

fig, ax = plt.subplots(1,2, squeeze=False)
ax[0,0].imshow(idat[0,0])
ax[0,1].imshow(idat_pe_pf.rss()[0,0])

This is much simpler and all of the magic is done in the background and we don't have to worry about it. Let's try it on the undersampled dataset.

## Reconstruction of undersampled data

In [ ]:
kdata_us = mrpro.data.KData.from_file(data_folder / Path('cart_t1_msense_integrated.mrd'), mrpro.data.traj_calculators.KTrajectoryCartesian())
direct_recon_us = mrpro.algorithms.reconstruction.DirectReconstruction(kdata_us)
idat_us = direct_recon_us(kdata_us)

fig, ax = plt.subplots(1,2, squeeze=False)
ax[0,0].imshow(idat[0,0])
ax[0,1].imshow(idat_us.rss()[0,0])

As expected we can see undersampling artefacts in the image. In order to get rid of them we can use an iterative SENSE algorithm. Of course for this regularly undersampled we could also use Cartesian SENSE unfolding but in MRpro we don't have that. 

Similary to the DirectReconstruction we can create an IterativeSENSEReconstruction and apply it to the undersampled KData. Give it a try and see if you can remove the undersampling artefacts. 

One important thing to keep in mind is, that this only works if the coil maps which we use, do not have any undersampling artefacts. Commonly we would get them from a fully sampled self-calibration reference lines in the centre of k-space or a separate coil sensitivity scan. Here we are going to cheat a bit and simply use the coil sensitivity maps from the fully sampled or the data with partial echo and partial Fourier. You can get the CsmData which is needed for the IterativeSENSEReconstruction by calling e.g. direct_recon_pe_pf.csm.

In [ ]:
kdata_us = mrpro.data.KData.from_file(data_folder / Path('cart_t1_msense_integrated.mrd'), mrpro.data.traj_calculators.KTrajectoryCartesian())
it_sense_recon = mrpro.algorithms.reconstruction.IterativeSENSEReconstruction(kdata_us, csm=direct_recon_pe_pf.csm)
idat_us = it_sense_recon(kdata_us)

fig, ax = plt.subplots(1,2, squeeze=False)
ax[0,0].imshow(idat[0,0])
ax[0,1].imshow(idat_us.rss()[0,0])